In [1]:
setwd('C:/Users/fleur/2021 P-SAT/주제분석/주제분석 2주차/데이터')

In [2]:
library(data.table)
library(tidyverse)
library(fastDummies)
library(car)
library(MLmetrics)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

√ ggplot2 3.3.3     √ purrr   0.3.4
√ tibble  3.0.4     √ dplyr   1.0.2
√ tidyr   1.1.2     √ stringr 1.4.0
√ readr   1.4.0     √ forcats 0.5.0

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::between()   masks data.table::between()
x dplyr::filter()    masks stats::filter()
x dplyr::first()     masks data.table::first()
x dplyr::lag()       masks stats::lag()
x dplyr::last()      masks data.table::last()
x purrr::transpose() masks data.table::transpose()

Warning message:
"package 'fastDummies' was built under R version 4.0.5"
Warning message:
"package 'car' was built under R version 4.0.4"
Loading required package: carData


Attaching package: 'car'


The following object is masked from 'package:dplyr':

    recode


The following object is masked from 'package:purrr':

    some


Wa

In [18]:
set.seed(613)

In [19]:
# 오즈비 출력 함수 정의
ORtable = function(x, digits = 7){
    suppressMessages(a <- confint(x))
    result = data.frame(exp(coef(x)), exp(a))
    result = round(result, digits)
    result = cbind(result, round(summary(x)$coefficient[,4],5))
    colnames(result) = c("OR", "2.5%", "97.%", "p-value")
    result
}

In [20]:
cat = fread('flt_train_cat.csv', data.table = FALSE)

In [21]:
cat %>% head()

,adoptionYN,age,season,neuterYN,sex,disease_cnt,positives,negatives,hair,grdp,economy,hospital_num,color
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,3,4,0,1,0,0,1,1,33275552,63007147,17,4
2,1,3,4,0,1,1,0,1,1,33275552,63007147,17,3
3,0,2,4,0,1,2,0,1,1,33275552,63007147,17,0
4,1,2,4,0,1,0,1,0,1,31765708,76346777,12,6
5,1,2,4,0,1,0,1,0,1,31765708,76346777,12,6
6,1,8,4,2,2,1,0,0,0,40023584,67504454,12,6


In [22]:
cat$adoptionYN = cat$adoptionYN %>% as.factor()
cat$season = cat$season %>% as.factor()
cat$neuterYN = cat$neuterYN %>% as.factor()
cat$sex = cat$sex %>% as.factor()
cat$hair = cat$hair %>% as.factor()
cat$color = cat$color %>% as.factor()

In [23]:
cat_cate = c('season', 'neuterYN', 'sex', 'hair', 'color')

In [24]:
cat_nume = c('age', 'disease_cnt', 'positives', 'negatives', 'grdp', 'economy', 'hospital_num')

In [25]:
cat_trainx = select(cat, -'adoptionYN')
cat_trainy = cat['adoptionYN']

In [26]:
cat_trainx %>% head()

,age,season,neuterYN,sex,disease_cnt,positives,negatives,hair,grdp,economy,hospital_num,color
,<int>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>,<dbl>,<dbl>,<int>,<fct>
1,3,4,0,1,0,0,1,1,33275552,63007147,17,4
2,3,4,0,1,1,0,1,1,33275552,63007147,17,3
3,2,4,0,1,2,0,1,1,33275552,63007147,17,0
4,2,4,0,1,0,1,0,1,31765708,76346777,12,6
5,2,4,0,1,0,1,0,1,31765708,76346777,12,6
6,8,4,2,2,1,0,0,0,40023584,67504454,12,6


In [27]:
cat_trainx_dum = dummy_cols(cat_trainx, select_columns = cat_cate, remove_first_dummy = TRUE)

In [28]:
cat_trainx_dum = select(cat_trainx_dum, -cat_cate)

In [29]:
cat_trainx_dum[cat_nume] = scale(cat_trainx_dum[cat_nume], center = TRUE, scale = TRUE)

In [30]:
cat_data = cbind(cat_trainy, cat_trainx_dum)

In [31]:
cat_data %>% head()

,adoptionYN,age,disease_cnt,positives,negatives,grdp,economy,hospital_num,season_2,season_3,...,sex_2,hair_1,hair_2,color_1,color_2,color_3,color_4,color_5,color_6,color_7
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,1.3145460,-0.70222841,-0.3783636,0.3740513,-0.26859438,-1.1095041,-0.6308720,0,0,...,0,1,0,0,0,0,1,0,0,0
2,1,1.3145460,-0.09656346,-0.3783636,0.3740513,-0.26859438,-1.1095041,-0.6308720,0,0,...,0,1,0,0,0,1,0,0,0,0
3,0,0.4669407,0.50910149,-0.3783636,0.3740513,-0.26859438,-1.1095041,-0.6308720,0,0,...,0,1,0,0,0,0,0,0,0,0
4,1,0.4669407,-0.70222841,1.4217553,-0.7218730,-0.31599116,0.4766053,-0.8388315,0,0,...,0,1,0,0,0,0,0,0,1,0
5,1,0.4669407,-0.70222841,1.4217553,-0.7218730,-0.31599116,0.4766053,-0.8388315,0,0,...,0,1,0,0,0,0,0,0,1,0
6,1,5.5525724,-0.09656346,-0.3783636,-0.7218730,-0.05676135,-0.5747650,-0.8388315,0,0,...,1,0,0,0,0,0,0,0,1,0


In [32]:
fit = glm(adoptionYN ~ ., cat_data, family = binomial)

In [33]:
vif(fit)

age  disease_cnt    positives    negatives         grdp      economy 
    1.275018     1.054801     1.021510     1.087400     1.217894     1.101854 
hospital_num     season_2     season_3     season_4   neuterYN_1   neuterYN_2 
    1.087225     1.810464     1.763087     1.377021     1.127338     1.090621 
       sex_1        sex_2       hair_1       hair_2      color_1      color_2 
    1.346307     1.378685     1.389110     1.505615     1.220439     1.146875 
     color_3      color_4      color_5      color_6      color_7 
    1.453414     1.514966     1.498244     1.479127     1.252847

다중공선성 없음, 독립성 가정 만족.

In [34]:
summary(fit)


Call:
glm(formula = adoptionYN ~ ., family = binomial, data = cat_data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5846  -0.9381  -0.6078   1.1084   2.7653  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)   0.8113463  0.0841306   9.644  < 2e-16 ***
age          -0.1914728  0.0199397  -9.603  < 2e-16 ***
disease_cnt  -0.2261039  0.0176365 -12.820  < 2e-16 ***
positives     0.3312302  0.0164771  20.102  < 2e-16 ***
negatives    -0.5120555  0.0195924 -26.135  < 2e-16 ***
grdp         -0.0633999  0.0178715  -3.548 0.000389 ***
economy       0.1078051  0.0160639   6.711 1.93e-11 ***
hospital_num  0.0007249  0.0156062   0.046 0.962954    
season_2     -0.8207019  0.0420693 -19.508  < 2e-16 ***
season_3     -0.5747225  0.0443069 -12.971  < 2e-16 ***
season_4     -0.0612476  0.0589225  -1.039 0.298590    
neuterYN_1    0.3288274  0.1223344   2.688 0.007189 ** 
neuterYN_2   -0.2880486  0.0376241  -7.656 1.92e-14 ***
sex_1        -

In [35]:
ORtable(fit)

,OR,2.5%,97.%,p-value
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),2.2509365,1.9089618,2.6548527,0.00000
age,0.8257421,0.7938166,0.8583625,0.00000
disease_cnt,0.7976352,0.7704071,0.8255573,0.00000
positives,1.3926804,1.3486989,1.4386930,0.00000
negatives,0.5992625,0.5765669,0.6225949,0.00000
grdp,0.9385681,0.9060558,0.9718297,0.00039
economy,1.1138306,1.0792872,1.1494406,0.00000
hospital_num,1.0007251,0.9705359,1.0317675,0.96295
season_2,0.4401226,0.4052609,0.4779218,0.00000


In [36]:
# 평가
test_cat = fread('flt_test_cat.csv', data.table = FALSE)

In [37]:
test_cat$adoptionYN = test_cat$adoptionYN %>% as.factor()

In [38]:
test_cat$season = test_cat$season %>% as.factor()
test_cat$neuterYN = test_cat$neuterYN %>% as.factor()
test_cat$sex = test_cat$sex %>% as.factor()
test_cat$hair = test_cat$hair %>% as.factor()
test_cat$color = test_cat$color %>% as.factor()

In [39]:
cat_testx = select(test_cat, -'adoptionYN')
cat_testy = test_cat['adoptionYN']

In [40]:
cat_testx %>% head()

,age,season,neuterYN,sex,disease_cnt,positives,negatives,hair,grdp,economy,hospital_num,color
,<int>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>,<dbl>,<dbl>,<int>,<fct>
1,1,4,0,0,2,0,0,1,33275552,63007147,17,4
2,3,4,0,0,0,0,1,1,33275552,63007147,17,4
3,2,4,0,0,0,0,0,1,98854601,75944548,24,4
4,3,4,0,1,0,2,0,2,31765708,76346777,12,7
5,1,4,2,1,0,1,0,0,38946656,76346777,39,0
6,1,4,0,1,4,0,1,1,33275552,63007147,17,0


In [41]:
cat_testx_dum = dummy_cols(cat_testx, select_columns = cat_cate, remove_first_dummy = TRUE)

In [42]:
cat_testx_dum = select(cat_testx_dum, -cat_cate)

In [43]:
cat_testx_dum[cat_nume] = scale(cat_testx_dum[cat_nume], center = TRUE, scale = TRUE)

In [44]:
cat_test = cbind(cat_testy, cat_testx_dum)

In [45]:
cat_test %>% head()

,adoptionYN,age,disease_cnt,positives,negatives,grdp,economy,hospital_num,season_2,season_3,...,sex_2,hair_1,hair_2,color_1,color_2,color_3,color_4,color_5,color_6,color_7
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0,-0.3892979,0.551606,-0.3848244,-0.7388560,-0.25909620,-1.1020430,-0.6284855,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0,1.3016403,-0.712268,-0.3848244,0.3588729,-0.25909620,-1.1020430,-0.6284855,0,0,...,0,1,0,0,0,0,1,0,0,0
3,0,0.4561712,-0.712268,-0.3848244,-0.7388560,1.80135373,0.4379490,-0.3286997,0,0,...,0,1,0,0,0,0,1,0,0,0
4,1,1.3016403,-0.712268,3.2798662,-0.7388560,-0.30653450,0.4858279,-0.8426181,0,0,...,0,0,1,0,0,0,0,0,0,1
5,1,-0.3892979,-0.712268,1.4475209,-0.7388560,-0.08091391,0.4858279,0.3136983,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,-0.3892979,1.815480,-0.3848244,0.3588729,-0.25909620,-1.1020430,-0.6284855,0,0,...,0,1,0,0,0,0,0,0,0,0


In [46]:
probabilities <- fit %>% predict(cat_test, type = "response")
y_pred <- ifelse(probabilities > 0.5, 1, 0)

In [47]:
F1_Score(cat_testy$adoptionYN, y_pred, positive = NULL)

[1] 0.7785402

In [48]:
Accuracy(cat_testy$adoptionYN, y_pred)

[1] 0.6914385